# Синхронизация с гит

In [ ]:
# --- БЛОК №1: НАЧАЛО РАБОТЫ (выполнять в каждой новой сессии) ---

from google.colab import drive, userdata
import os

# 1. Монтируем Google Drive
drive.mount('/content/drive')

# 2. Получаем секретный токен
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')

# 3. Указываем ваше имя и email (как на GitHub)
GIT_USERNAME = "kirichich1" # <-- Ваше имя пользователя
GIT_EMAIL = "kirichich@bk.ru" # <-- Ваша почта

# 4. !!! ПРЕДСТАВЛЯЕМСЯ СИСТЕМЕ GIT !!!
# Эта конфигурация будет действовать до конца текущей сессии
!git config --global user.name "{GIT_USERNAME}"
!git config --global user.email "{GIT_EMAIL}"

# 5. Переходим в директорию проекта
PROJECT_PATH = "/content/drive/MyDrive/GitHub/no2_prediction_pipeline"
%cd {PROJECT_PATH}

# 6. (Рекомендуется) Скачиваем последние изменения с GitHub на случай, если вы работали с другого устройства
!git pull

# Импорт библиотек и загрузка данных

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [ ]:
file_path = '/content/drive/MyDrive/GitHub/no2_prediction_pipeline/data/processed/Surgut_data_2019_to_2024_COMBINED_WITH_AAI.csv'

In [ ]:
df = pd.read_csv(file_path)

# Начальная очистка и подготовка данных

In [ ]:
# --- Начальная очистка и подготовка данных (ИСПРАВЛЕННАЯ ВЕРСИЯ) ---

# Преобразуем колонку 'date' в формат datetime для дальнейшей работы
df['date'] = pd.to_datetime(df['date'])

# Создаем целевую переменную и удаляем дубликат, если необходимо
if 'no2_umol_m2' not in df.columns:
    df['no2_umol_m2'] = df['no2_trop_mean'] * 1e6
if 'no2_trop_mean' in df.columns:
    df = df.drop('no2_trop_mean', axis=1)

# Заполняем редкие пропуски в признаках
# Важно: Сначала заполняем пропуски в исходном df
median_cloud = df['cloud_frac_mean'].median()
df['cloud_frac_mean'].fillna(median_cloud, inplace=True)

# !!! Ключевой шаг: работаем только с реальными данными NO2
### ИЗМЕНЕНИЕ ЗДЕСЬ: добавили .copy() ###
df_processed = df.dropna(subset=['no2_umol_m2']).copy()

print(f"Количество строк после удаления пропусков в NO2: {len(df_processed)}")

# Feature Engineering

In [ ]:
# --- Feature Engineering (ВЕРСИЯ: Winter Interaction) ---

# --- ШАГ 3.1: Базовые признаки ---
print("Создание базовых признаков...")
df_processed['year'] = df_processed['date'].dt.year

# Вектор ветра
df_processed['wind_speed'] = np.sqrt(df_processed['u_wind_10m']**2 + df_processed['v_wind_10m']**2)
df_processed['wind_direction'] = (270 - np.arctan2(df_processed['v_wind_10m'], df_processed['u_wind_10m']) * 180 / np.pi) % 360

# Календарь
df_processed['month'] = df_processed['date'].dt.month
df_processed['day_of_year'] = df_processed['date'].dt.dayofyear
df_processed['day_of_week'] = df_processed['date'].dt.dayofweek
df_processed['week_of_year'] = df_processed['date'].dt.isocalendar().week.astype(int)

# --- ФИЗИЧЕСКИЕ ПРИЗНАКИ ---

# 1. HDD (Нагрузка на отопление)
df_processed['HDD'] = (18 - df_processed['temperature_celsius']).clip(lower=0)

# 2. Индекс Застоя (Stagnation)
df_processed['stagnation_index'] = df_processed['pressure_hpa'] / (df_processed['wind_speed'] + 0.1)

# 3. !!! НОВОЕ: Зимняя ловушка (Взаимодействие холода и застоя) !!!
# Чем холоднее (высокий HDD) И чем сильнее застой, тем выше этот коэффициент.
df_processed['winter_stagnation'] = df_processed['HDD'] * df_processed['stagnation_index']

# --- ШАГ 3.2: Лаги и Тренды ---
print("\nСоздание лагов и скользящих средних...")
df_processed = df_processed.sort_values(by='date')
grouped = df_processed.groupby('year')

# Лаги
df_processed['no2_lag_1'] = grouped['no2_umol_m2'].shift(1)
df_processed['wind_speed_lag_1'] = grouped['wind_speed'].shift(1)
df_processed['temperature_lag_1'] = grouped['temperature_celsius'].shift(1)
df_processed['aai_lag_1'] = grouped['aai_mean'].shift(1)

# Скользящие средние
df_processed['no2_roll_mean_3'] = grouped['no2_umol_m2'].shift(1).rolling(window=3).mean()
df_processed['no2_roll_mean_7'] = grouped['no2_umol_m2'].shift(1).rolling(window=7).mean()
df_processed['wind_roll_mean_3'] = grouped['wind_speed'].shift(1).rolling(window=3).mean()

# Delta Wind
df_processed['delta_wind'] = df_processed['wind_speed'] - df_processed['wind_speed_lag_1']

# --- ШАГ 3.3: Очистка ---
initial_rows = len(df_processed)
roll_columns = ['no2_roll_mean_3', 'no2_roll_mean_7', 'wind_roll_mean_3', 'no2_lag_1']
df_processed.dropna(subset=roll_columns, inplace=True)

print(f"Итоговое количество строк: {len(df_processed)}")

# Разделение данных на обучающую и тестовую выборки

In [ ]:
# Обучаемся на всех данных ДО 2024 года
train_data = df_processed[df_processed['year'] < 2024]
# Тестируем на ПОЛНОМ 2024 годе
test_data = df_processed[df_processed['year'] == 2024]

In [ ]:
print(f"\nРазмер обучающей выборки (2019-2023): {len(train_data)} строк")
print(f"Размер тестовой выборки (2024): {len(test_data)} строк")

# Определение признаков (X) и целевой переменной (y)

In [ ]:
# --- ОПРЕДЕЛЕНИЕ ПРИЗНАКОВ ---
train_data = df_processed[df_processed['year'] < 2024]
test_data = df_processed[df_processed['year'] == 2024]

# Исходные целевые переменные
y_train_raw = train_data['no2_umol_m2']
y_test = test_data['no2_umol_m2']
dates_test = test_data['date']

# !!! ИСПРАВЛЕНИЕ ОШИБКИ !!!
# 1. Убираем отрицательные значения (шум спутника), которые ломают логарифм
y_train_raw = y_train_raw.clip(lower=0)

# 2. Логарифмируем целевую переменную
# log1p подходит идеально, так как log1p(0) = 0
y_train_log = np.log1p(y_train_raw)

# Проверка на NaN после логарифмирования (для спокойствия)
if y_train_log.isna().any():
    print("Внимание! В y_train_log все еще есть NaN. Заполняем нулями.")
    y_train_log = y_train_log.fillna(0)

features = [
    'cloud_frac_mean', 'temperature_celsius', 'pressure_hpa',
    'wind_speed', 'wind_direction', 'aai_mean',
    'month', 'day_of_year', 'day_of_week', 'week_of_year',

    'HDD',
    'stagnation_index',
    'winter_stagnation',
    'delta_wind',

    'no2_lag_1', 'wind_speed_lag_1', 'temperature_lag_1', 'aai_lag_1',
    'no2_roll_mean_3', 'no2_roll_mean_7', 'wind_roll_mean_3'
]

X_train = train_data[features]
X_test = test_data[features]

print("Размер y_train_log:", len(y_train_log))
print("Количество NaN в y_train_log:", y_train_log.isna().sum())

# Инициализация и обучение модели RandomForestRegressor

In [ ]:
# --- ОБУЧЕНИЕ МОДЕЛИ ---
model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1, oob_score=True)

print("\nОбучение модели на log(NO2)...")
model.fit(X_train, y_train_log)  # <-- Обучаем на логарифмах!
print("Модель обучена.")

# Предсказание и оценка качества модели

In [ ]:
# Получаем предсказание в логарифмах
y_pred_log = model.predict(X_test)

# !!! ВАЖНО: Возвращаем предсказания в обычный масштаб (экспонента) !!!
y_pred = np.expm1(y_pred_log)

# --- ОЦЕНКА ---
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("\n--- Оценка качества (с Log-transform) ---")
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R^2: {r2:.3f}")

# Визуализация результатов

In [ ]:
plt.style.use('seaborn-v0_8-whitegrid')
plt.figure(figsize=(15, 7))
plt.plot(dates_test, y_test.values, label='Реальные значения (2024)', color='blue', alpha=0.7)
plt.plot(dates_test, y_pred, label='Предсказанные значения (2024)', color='red', linestyle='--')
plt.title('Прогноз на 2024 год: Сравнение реальных и предсказанных значений NO2')
plt.xlabel('Дата')
plt.ylabel('Концентрация NO2 (мкмоль/м^2)')
plt.legend()
plt.show()

# Анализ важности признаков

In [ ]:
feature_importances = pd.DataFrame(model.feature_importances_, index=features, columns=['importance'])
feature_importances = feature_importances.sort_values('importance', ascending=False)

print("\n--- Важность признаков для модели ---")
print(feature_importances)

In [ ]:
df[df['no2_umol_m2'] > 100]

# Коммит для гит

In [ ]:
# --- БЛОК №2: СОХРАНЕНИЕ РЕЗУЛЬТАТОВ (в конце работы) ---

# 1. Проверяем статус (опционально)
!git status

# 2. ДОБАВЛЯЕМ файлы в "коробку" для отправки. Это то, что вы пропустили.
!git add .

# 3. ПОДПИСЫВАЕМ "коробку" (делаем коммит)
!git commit -m "Add winter_stagnation interaction and use Log-Target transformation"

# 4. ОТПРАВЛЯЕМ "коробку" на GitHub
!git push